### This notebook follows ``ors_iteration_add_rentalDistance.ipynb`` which computed distance/time from each property to all places and CBD. Now we only want the closest distances.

In [1]:
import pandas as pd
import sys
import os
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, '../../scripts/')
from add_distance import get_min_distance_time

In [3]:
if not os.path.exists('../../data/distance'):
    os.makedirs('../../data/distance')

In [4]:
YEARS = [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
for year in YEARS:
    distance_df = pd.read_csv(f"../../data/featured/{year}_distance_rental_place.csv")
    addr_month_pair = len(distance_df.drop_duplicates(subset=['address', 'month']))
    # Check original number of unique property-month pairs
    print(f"Originally {addr_month_pair} address-month pairs, shape = {distance_df.shape}")
    min_distance_df = get_min_distance_time(distance_df, year)

Originally 10940 address-month pairs, shape = (153905, 25)
Updated 10940 address-month pairs, shape = (34702, 22)
Originally 12088 address-month pairs, shape = (195828, 25)
Updated 12088 address-month pairs, shape = (40289, 22)
Originally 12490 address-month pairs, shape = (203387, 25)
Updated 12490 address-month pairs, shape = (41674, 22)
Originally 14751 address-month pairs, shape = (292284, 25)
Updated 14751 address-month pairs, shape = (49663, 22)
Originally 16708 address-month pairs, shape = (330166, 25)
Updated 16708 address-month pairs, shape = (56098, 22)
Originally 19045 address-month pairs, shape = (373152, 25)
Updated 19045 address-month pairs, shape = (63996, 22)
Originally 20876 address-month pairs, shape = (404404, 25)
Updated 20876 address-month pairs, shape = (70066, 22)
Originally 20479 address-month pairs, shape = (412000, 25)
Updated 20479 address-month pairs, shape = (68226, 22)
Originally 24945 address-month pairs, shape = (486159, 25)
Updated 24945 address-month p